#Model Inference API
Описание
В этом модуле реализованы функции для инференса (предсказания) ключевых HR-показателей:

Зарплата по вакансии (predict_salary)

Грейд (junior/middle/senior/lead) (predict_grade)

Функции полностью воспроизводят пайплайн, использованный при обучении моделей: учитывают числовые, категориальные и текстовые признаки, а также эмбеддинги описания.

In [4]:
%cd /content/drive/MyDrive/hh-hr-bot

/content/drive/MyDrive/hh-hr-bot


In [22]:
# src/bot/model_inference.py
%%writefile src/bot/model_inference.py

import os
import joblib
import pickle
import numpy as np
from scipy import sparse
from sentence_transformers import SentenceTransformer

MODEL_DIR = '/content/drive/MyDrive/hh-hr-bot/models'

SALARY_MODEL_PATH = os.path.join(MODEL_DIR, 'salary_lgbm_model.pkl')
GRADE_MODEL_PATH = os.path.join(MODEL_DIR, 'grade_rf_model.joblib')
TFIDF_DESC_PATH = os.path.join(MODEL_DIR, 'tfidf_desc.pkl')
TFIDF_TITLE_PATH = os.path.join(MODEL_DIR, 'tfidf_title.pkl')
OHE_CATS_PATH = os.path.join(MODEL_DIR, 'ohe_cats.pkl')

salary_model = joblib.load(SALARY_MODEL_PATH)
grade_model = joblib.load(GRADE_MODEL_PATH)

with open(TFIDF_DESC_PATH, 'rb') as f:
    tfidf_desc = pickle.load(f)
with open(TFIDF_TITLE_PATH, 'rb') as f:
    tfidf_title = pickle.load(f)
with open(OHE_CATS_PATH, 'rb') as f:
    ohe = pickle.load(f)

minilm_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

def prepare_features_full(features: dict):
    """ Полный пайплайн для salary (668 признаков) """
    num_feats = np.array([
        features.get('area_id', 0),
        features.get('desc_len', 0),
        features.get('desc_words', 0),
        features.get('title_len', 0),
        features.get('num_skills', 0),
        features.get('exp_junior', 0),
        features.get('exp_middle', 0),
        features.get('exp_senior', 0),
        features.get('exp_lead', 0)
    ]).reshape(1, -1)

    desc = features.get('description', "")
    tfidf_desc_vec = tfidf_desc.transform([desc])
    title = features.get('title', "")
    tfidf_title_vec = tfidf_title.transform([title])
    area_id = str(features.get('area_id', ""))
    salary_currency = str(features.get('salary_currency', "RUR"))
    ohe_cats_vec = ohe.transform([[area_id, salary_currency]])
    emb_vec = minilm_model.encode([desc])  # (1, 384)
    X = sparse.hstack([
        num_feats, tfidf_desc_vec, tfidf_title_vec, ohe_cats_vec, emb_vec
    ]).tocsr()
    return X

def prepare_features_emb_only(features: dict):
    """ Только эмбеддинг по описанию для grade-классификатора (384 признака) """
    desc = features.get('description', "")
    emb_vec = minilm_model.encode([desc])  # (1, 384)
    return emb_vec

def predict_salary(features: dict) -> float:
    X = prepare_features_full(features)
    salary_pred = salary_model.predict(X)[0]
    return float(salary_pred)

def predict_grade(features: dict) -> int:
    X = prepare_features_emb_only(features)
    grade_pred = grade_model.predict(X)[0]
    return int(grade_pred)

def predict_salary_response(features: dict) -> dict:
    value = predict_salary(features)
    return {
        "salary": int(value),
        "currency": features.get("salary_currency", "RUR")
    }

def predict_grade_response(features: dict) -> dict:
    code = predict_grade(features)
    grade_map = {0: "junior", 1: "middle", 2: "senior", 3: "lead"}
    return {
        "grade_code": int(code),
        "grade_label": grade_map.get(code, "unknown")
    }

#if __name__ == "__main__":
#    test_features = {
#        'area_id': 3,
#        'desc_len': 1400,
#        'desc_words': 120,
#        'title_len': 30,
#       'num_skills': 4,
#      'exp_junior': 0,
#        'exp_middle': 1,
#        'exp_senior': 0,
#        'exp_lead': 0,
#        'description': "Python-разработчик, опыт с Django и PostgreSQL, удалёнка.",
#        'title': "Python developer",
#        'salary_currency': "RUR"
#    }
#    print("Зарплата:", predict_salary(test_features))
#    print("Грейд:", predict_grade(test_features))



Overwriting src/bot/model_inference.py



#market_analytics API

Аналитические функции для HR-бота и автоматизации анализа рынка труда.

Содержит:
- top_5_skills: топ-5 востребованных навыков с фильтрацией по профессии, региону и грейду (опционально), а также средней зарплатой по вакансиям с этим навыком.
- compare_vacancy_to_market: сравнение вакансии с рынком по зарплате и ключевым навыкам (по аналогичным вакансиям из базы).

Все функции работают напрямую с базой hh.duckdb_3000 и могут использоваться в ботах, API, скриптах и презентациях.
"""


In [62]:
# src/bot/market_analytics.py
%%writefile src/bot/market_analytics.py
import duckdb

# Подключение к базе (укажи свой путь, если другой)
con = duckdb.connect('/content/drive/MyDrive/hh-hr-bot/data/hh.duckdb_3000')

# Карта грейдов для удобной фильтрации по “человеческим” грейдам
grade_map = {
    'Нет опыта': 0,          # junior
    'От 1 года до 3 лет': 1, # middle
    'От 3 до 6 лет': 2,      # senior
    'Более 6 лет': 3         # lead
}

def top_5_skills(title=None, area_id=None, grade=None):
    """
    Возвращает топ-5 навыков по частоте и средней зарплате.
    Фильтры:
      - title (str): фильтр по профессии (подстрока в названии вакансии)
      - area_id (int): фильтр по региону
      - grade (int/str): фильтр по грейду (0-3 или "Нет опыта", ...)
    """
    # Преобразуем grade к строке на русском, если нужно
    experience_hh = None
    if grade is not None:
        if isinstance(grade, int):
            for k, v in grade_map.items():
                if v == grade:
                    experience_hh = k
                    break
        elif grade in grade_map:
            experience_hh = grade
        else:
            raise ValueError("grade должен быть int (0-3) или одним из: " + ", ".join(grade_map.keys()))

    query = """
    SELECT
        vs.skill_name,
        COUNT(*) AS frequency,
        ROUND(AVG(vp.salary_rub), 0) AS mean_salary
    FROM vacancy_skill vs
    JOIN vacancy v ON vs.vacancy_id = v.id
    JOIN vacancy_proc vp ON v.id = vp.id
    WHERE 1=1
    """
    params = []
    if title:
        query += " AND LOWER(v.title) LIKE ?"
        params.append(f"%{title.lower()}%")
    if area_id is not None:
        query += " AND v.area_id = ?"
        params.append(area_id)
    if experience_hh is not None:
        query += " AND v.experience_hh = ?"
        params.append(experience_hh)
    query += """
    GROUP BY vs.skill_name
    ORDER BY frequency DESC
    LIMIT 5
    """
    return con.execute(query, params).df()

def compare_vacancy_to_market(vac: dict):
    """
    Сравнивает переданную вакансию (dict) с рынком аналогичных вакансий:
      - title (str)
      - area_id (int)
      - experience_hh (str, например, 'От 1 года до 3 лет')
      - skills (list или строка через ;)
      - salary_rub (int/float, опционально)
    Возвращает текстовый отчёт с анализом зарплаты и навыков.
    """
    title = vac['title']
    area_id = vac['area_id']
    experience_hh = vac['experience_hh']
    skills = vac['skills']
    salary = vac.get('salary_rub', None)

    # === 1. Формируем "аналогичный рынок" вакансий ===
    query = """
    SELECT v.id, v.title, v.area_id, v.experience_hh, vp.salary_rub
    FROM vacancy v
    JOIN vacancy_proc vp ON v.id = vp.id
    WHERE v.area_id = ?
      AND v.experience_hh = ?
      AND LOWER(v.title) LIKE ?
      AND vp.salary_rub IS NOT NULL
    """
    params = [area_id, experience_hh, f"%{title.lower()}%"]
    df_market = con.execute(query, params).fetchdf()

    if df_market.empty:
        return "Нет сопоставимых вакансий для сравнения."

    # === 2. Медианная зарплата по рынку ===
    market_salary = df_market['salary_rub'].median()
    result = ""
    if salary:
        diff = salary - market_salary
        perc = round(100 * diff / market_salary, 1)
        result += (
            f"Ваша вакансия: {title} ({experience_hh}, регион {area_id})\n"
            f"Зарплата: {int(salary)} руб.\n"
            f"Медиана рынка: {int(market_salary)} руб.\n"
            f"Отклонение: {'+' if diff > 0 else ''}{int(diff)} руб. "
            f"({'+' if perc > 0 else ''}{perc}%)\n"
        )
    else:
        result += (
            f"Ваша вакансия: {title} ({experience_hh}, регион {area_id})\n"
            f"Медиана зарплаты по рынку: {int(market_salary)} руб.\n"
        )

    # === 3. Анализ навыков ===
    # BinderError fix: явно приводим id к BIGINT
    q_skills = """
    SELECT vs.skill_name
    FROM vacancy_skill vs
    WHERE vs.vacancy_id IN (
        SELECT CAST(v.id AS BIGINT)
        FROM vacancy v
        WHERE v.area_id = ?
          AND v.experience_hh = ?
          AND LOWER(v.title) LIKE ?
    )
    """
    skills_df = con.execute(q_skills, params).fetchdf()

    if not skills_df.empty:
        top_skills = (
            skills_df['skill_name']
            .value_counts()
            .head(10)
            .index
            .tolist()
        )
        if isinstance(skills, str):
            your_skills = [s.strip() for s in skills.split(';') if s.strip()]
        else:
            your_skills = list(skills)
        common = [s for s in your_skills if s in top_skills]
        unique = [s for s in your_skills if s not in top_skills]
        missing = [s for s in top_skills if s not in your_skills]
        result += (
            f"\nВаши навыки: {', '.join(your_skills)}\n"
            f"Топ-10 популярных навыков на рынке: {', '.join(top_skills)}\n"
        )
        if unique:
            result += f"Уникальные для вас навыки (редко встречаются): {', '.join(unique)}\n"
        if missing:
            result += f"Рекомендуется добавить популярные навыки: {', '.join(missing)}\n"
    return result


Overwriting src/bot/market_analytics.py


In [29]:
import duckdb

# Подключение к базе
con = duckdb.connect('/content/drive/MyDrive/hh-hr-bot/data/hh.duckdb_3000')

# Карта грейдов для удобства фильтрации (если нужно фильтровать по коду)
grade_map = {
    'Нет опыта': 0,          # junior
    'От 1 года до 3 лет': 1, # middle
    'От 3 до 6 лет': 2,      # senior
    'Более 6 лет': 3         # lead
}

def top_5_skills(title=None, area_id=None, grade=None):
    """
    Топ-5 навыков по частоте и средней зарплате. Фильтры:
    - title: подстрока в названии профессии
    - area_id: регион
    - grade: int-код (0,1,2,3) или текст на русском ("Нет опыта", ...)
    """
    # Если grade задан как int — подбираем строку для фильтра
    experience_hh = None
    if grade is not None:
        # Можно передать как int или как строку на русском
        if isinstance(grade, int):
            for k, v in grade_map.items():
                if v == grade:
                    experience_hh = k
                    break
        elif grade in grade_map:
            experience_hh = grade
        else:
            raise ValueError("grade должен быть int (0-3) или одним из: " + ", ".join(grade_map.keys()))

    query = """
    SELECT
        vs.skill_name,
        COUNT(*) AS frequency,
        ROUND(AVG(vp.salary_rub), 0) AS mean_salary
    FROM vacancy_skill vs
    JOIN vacancy v ON vs.vacancy_id = v.id
    JOIN vacancy_proc vp ON v.id = vp.id
    WHERE 1=1
    """
    params = []
    if title:
        query += " AND LOWER(v.title) LIKE ?"
        params.append(f"%{title.lower()}%")
    if area_id is not None:
        query += " AND v.area_id = ?"
        params.append(area_id)
    if experience_hh is not None:
        query += " AND v.experience_hh = ?"
        params.append(experience_hh)
    query += """
    GROUP BY vs.skill_name
    ORDER BY frequency DESC
    LIMIT 5
    """
    return con.execute(query, params).df()

# Примеры вызова:
# Топ-5 по Python, регион 3, грейд "senior" (можно и по коду, и по строке)
print(top_5_skills(title="Инженер", area_id=2, grade=1))             # senior
print(top_5_skills(title="Python", area_id=3, grade="От 3 до 6 лет"))




               skill_name  frequency  mean_salary
0  Проектная документация          2     317500.0
1          Проектирование          1     135000.0
2          Разработка РЭА          1     135000.0
3                  Python          1          NaN
4                    VoIP          1     120000.0
Empty DataFrame
Columns: [skill_name, frequency, mean_salary]
Index: []


In [32]:
import duckdb

# Подключение к базе DuckDB (путь подставь свой)
con = duckdb.connect('/content/drive/MyDrive/hh-hr-bot/data/hh.duckdb_3000')

def compare_vacancy_to_market(vac: dict):
    """
    Сравнивает переданную вакансию (dict) с рынком аналогичных вакансий:
    - title: профессия (str)
    - area_id: регион (int)
    - experience_hh: грейд (str, например, 'От 1 года до 3 лет')
    - skills: список или строка (навыки через ;)
    - salary_rub: (float/int, желательно для отчёта, но не обязательно)

    Возвращает текстовый отчёт с анализом зарплаты и навыков.
    """

    # Извлекаем данные из входного словаря
    title = vac['title']
    area_id = vac['area_id']
    experience_hh = vac['experience_hh']
    skills = vac['skills']
    salary = vac.get('salary_rub', None)  # Может отсутствовать

    # === 1. Формируем аналогичный "рынок" вакансий ===
    # Выбираем из базы вакансии с тем же регионом, грейдом и похожим названием,
    # которые имеют зарплату (salary_rub)
    query = """
    SELECT v.id, v.title, v.area_id, v.experience_hh, vp.salary_rub
    FROM vacancy v
    JOIN vacancy_proc vp ON v.id = vp.id
    WHERE v.area_id = ?
      AND v.experience_hh = ?
      AND LOWER(v.title) LIKE ?
      AND vp.salary_rub IS NOT NULL
    """
    params = [area_id, experience_hh, f"%{title.lower()}%"]
    df_market = con.execute(query, params).fetchdf()

    # Если подходящих вакансий нет — сразу возвращаем сообщение
    if df_market.empty:
        return "Нет сопоставимых вакансий для сравнения."

    # === 2. Считаем медиану зарплаты по рынку ===
    market_salary = df_market['salary_rub'].median()

    # Формируем текст отчёта о зарплате
    result = ""
    if salary:
        # Если у пользователя есть своя зарплата — сравниваем с рынком
        diff = salary - market_salary
        perc = round(100 * diff / market_salary, 1)
        result += (
            f"Ваша вакансия: {title} ({experience_hh}, регион {area_id})\n"
            f"Зарплата: {int(salary)} руб.\n"
            f"Медиана рынка: {int(market_salary)} руб.\n"
            f"Отклонение: {'+' if diff > 0 else ''}{int(diff)} руб. "
            f"({'+' if perc > 0 else ''}{perc}%)\n"
        )
    else:
        # Если зарплата не указана — просто сообщаем медиану рынка
        result += (
            f"Ваша вакансия: {title} ({experience_hh}, регион {area_id})\n"
            f"Медиана зарплаты по рынку: {int(market_salary)} руб.\n"
        )

    # === 3. Анализ навыков ===
    # Находим топ-10 навыков среди аналогичных вакансий (по id)
    q_skills = """
    SELECT vs.skill_name
    FROM vacancy_skill vs
    WHERE vs.vacancy_id IN (
        SELECT CAST(v.id AS BIGINT)
        FROM vacancy v
        WHERE v.area_id = ?
          AND v.experience_hh = ?
          AND LOWER(v.title) LIKE ?
    )
    """
    skills_df = con.execute(q_skills, params).fetchdf()

    if not skills_df.empty:
        # Находим 10 самых частых навыков на рынке (по аналогичным вакансиям)
        top_skills = (
            skills_df['skill_name']
            .value_counts()
            .head(10)
            .index
            .tolist()
        )
        # Приводим твои навыки к списку
        if isinstance(skills, str):
            your_skills = [s.strip() for s in skills.split(';') if s.strip()]
        else:
            your_skills = list(skills)
        # Сравниваем: какие твои навыки есть на рынке (common), какие уникальны (unique)
        common = [s for s in your_skills if s in top_skills]
        unique = [s for s in your_skills if s not in top_skills]
        # Какие популярные навыки рынка отсутствуют у тебя (missing)
        missing = [s for s in top_skills if s not in your_skills]
        # Добавляем информацию о навыках в отчёт
        result += (
            f"\nВаши навыки: {', '.join(your_skills)}\n"
            f"Топ-10 популярных навыков на рынке: {', '.join(top_skills)}\n"
        )
        if unique:
            result += f"Уникальные для вас навыки (редко встречаются): {', '.join(unique)}\n"
        if missing:
            result += f"Рекомендуется добавить популярные навыки: {', '.join(missing)}\n"

    # Возвращаем готовый отчёт
    return result

# Пример использования:
my_vac = {
    'title': 'Инженер',
    'area_id': 2,
    'experience_hh': 'От 1 года до 3 лет',
    'skills': ['Python', 'SQL', 'Django', 'English'],
    'salary_rub': 120000
}
print(compare_vacancy_to_market(my_vac))


Ваша вакансия: Инженер (От 1 года до 3 лет, регион 2)
Зарплата: 120000 руб.
Медиана рынка: 142500 руб.
Отклонение: -22500 руб. (-15.8%)

Ваши навыки: Python, SQL, Django, English
Топ-10 популярных навыков на рынке: Проектная документация, Linux, VoIP, Web тестирование, Python, Разработка технических заданий, Разработка РЭА, DesignerСхемотехника электронного оборудования, Чтение электросхем, Проектирование
Уникальные для вас навыки (редко встречаются): SQL, Django, English
Рекомендуется добавить популярные навыки: Проектная документация, Linux, VoIP, Web тестирование, Разработка технических заданий, Разработка РЭА, DesignerСхемотехника электронного оборудования, Чтение электросхем, Проектирование



In [34]:
pip install python-telegram-bot --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.3/702.3 kB 9.6 MB/s eta 0:00:00


In [13]:
%%writefile src/bot/bot.py
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes,MessageHandler, filters, ConversationHandler
from model_inference import predict_salary
from telegram.constants import ParseMode
from market_analytics import top_5_skills, compare_vacancy_to_market
import pandas as pd


SALARY_INPUT = 1

SKILLS_INPUT = 2

ANALYZE_INPUT = 3

TELEGRAM_TOKEN =

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Привет! Я HR-бот. Могу предсказать зарплату, грейд, показать топ навыков и сравнить вакансию с рынком.\n"
        "Команды:\n"
        "/salary — прогноз зарплаты\n"
        "/grade — определение грейда\n"
        "/skills — топ-5 востребованных навыков\n"
        "/analyze — сравнить вакансию с рынком\n"
        "\nДля справки введите /help"
    )
async def salary_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Введите параметры вакансии через запятую в формате:\n"
        "area_id, desc_len, desc_words, title_len, num_skills, exp_junior, exp_middle, exp_senior, exp_lead, description, title, salary_currency\n\n"
        "Пример:\n"
        "3, 1200, 100, 30, 4, 0, 1, 0, 0, Python-разработчик, Python developer, RUR"
    )
    return SALARY_INPUT

async def handle_salary_input(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        text = update.message.text.strip()
        parts = [p.strip() for p in text.split(",")]
        if len(parts) < 12:
            await update.message.reply_text("Ошибка: введено слишком мало параметров. Проверьте формат.")
            return SALARY_INPUT

        features = {
            'area_id': int(parts[0]),
            'desc_len': int(parts[1]),
            'desc_words': int(parts[2]),
            'title_len': int(parts[3]),
            'num_skills': int(parts[4]),
            'exp_junior': int(parts[5]),
            'exp_middle': int(parts[6]),
            'exp_senior': int(parts[7]),
            'exp_lead': int(parts[8]),
            'description': parts[9],
            'title': parts[10],
            'salary_currency': parts[11]
        }
        salary = predict_salary(features)
        await update.message.reply_text(
            f"Прогнозируемая зарплата: <b>{int(salary):,} {features['salary_currency']}</b>",
            parse_mode='HTML'
        )
        return ConversationHandler.END
    except Exception as e:
        await update.message.reply_text(f"Ошибка: {str(e)}")
        return SALARY_INPUT

async def skills_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Введите параметры через запятую в формате:\n"
        "title, area_id, grade (например: Python developer, 3, 2)\n"
        "Где grade: 0 — junior, 1 — middle, 2 — senior, 3 — lead\n"
        "Можно оставить grade или area_id пустым, если хотите искать по всем регионам/грейдам."
    )
    return SKILLS_INPUT

async def handle_skills_input(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        text = update.message.text.strip()
        parts = [p.strip() for p in text.split(",")]

        title = parts[0] if len(parts) > 0 and parts[0] else None
        area_id = int(parts[1]) if len(parts) > 1 and parts[1] else None
        grade = int(parts[2]) if len(parts) > 2 and parts[2] else None

        result = top_5_skills(title=title, area_id=area_id, grade=grade)

        if result.empty:
            await update.message.reply_text("Нет данных для указанных параметров.")
        else:
            msg = "Топ-5 востребованных навыков:\n"
            for i, row in result.iterrows():
                msg += f"{i+1}. {row['skill_name']} — {row['frequency']} вакансий, средняя зарплата: {int(row['mean_salary']) if not pd.isna(row['mean_salary']) else '-'}\n"
            await update.message.reply_text(msg)
        return ConversationHandler.END
    except Exception as e:
        await update.message.reply_text(f"Ошибка: {str(e)}")
        return SKILLS_INPUT

async def analyze_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Введите параметры через запятую в формате:\n"
        "title, area_id, experience_hh, skills, salary_rub\n"
        "Пример:\n"
        "Python developer, 3, От 1 года до 3 лет, Python;SQL;Django;English, 120000\n"
        "Если не хотите указывать зарплату — оставьте поле пустым (последний параметр)."
    )
    return ANALYZE_INPUT

async def handle_analyze_input(update: Update, context: ContextTypes.DEFAULT_TYPE):
    try:
        text = update.message.text.strip()
        parts = [p.strip() for p in text.split(",")]

        if len(parts) < 4:
            await update.message.reply_text("Ошибка: слишком мало параметров. Проверьте формат.")
            return ANALYZE_INPUT

        title = parts[0]
        area_id = int(parts[1])
        experience_hh = parts[2]
        skills = parts[3]
        salary_rub = int(parts[4]) if len(parts) > 4 and parts[4] else None

        vacancy = {
            'title': title,
            'area_id': area_id,
            'experience_hh': experience_hh,
            'skills': skills,
            'salary_rub': salary_rub
        }
        result = compare_vacancy_to_market(vacancy)
        await update.message.reply_text(result)
        return ConversationHandler.END
    except Exception as e:
        await update.message.reply_text(f"Ошибка: {str(e)}")
        return ANALYZE_INPUT

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Напиши /salary, /grade, /skills или /analyze для получения аналитики по рынку труда.")

def main():

    app = ApplicationBuilder().token(TELEGRAM_TOKEN).build()

    conv_salary = ConversationHandler(
        entry_points=[CommandHandler("salary", salary_command)],
        states={
            SALARY_INPUT: [MessageHandler(filters.TEXT & ~filters.COMMAND, handle_salary_input)],
        },
        fallbacks=[CommandHandler("cancel", lambda update, context: update.message.reply_text("Операция отменена."))]
    )

    conv_skills = ConversationHandler(
        entry_points=[CommandHandler("skills", skills_command)],
        states={
            SKILLS_INPUT: [MessageHandler(filters.TEXT & ~filters.COMMAND, handle_skills_input)],
        },
        fallbacks=[CommandHandler("cancel", lambda update, context: update.message.reply_text("Операция отменена."))]
    )
    conv_analyze = ConversationHandler(
        entry_points=[CommandHandler("analyze", analyze_command)],
        states={
            ANALYZE_INPUT: [MessageHandler(filters.TEXT & ~filters.COMMAND, handle_analyze_input)],
        },
        fallbacks=[CommandHandler("cancel", lambda update, context: update.message.reply_text("Операция отменена."))]
     )


    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("help", help_command))
    app.add_handler(conv_salary)
    app.add_handler(conv_skills)
    app.add_handler(conv_analyze)


    print("Бот запущен!")
    app.run_polling()

if __name__ == "__main__":
    main()


Overwriting src/bot/bot.py


In [12]:
!python src/bot/bot.py

2025-05-30 01:27:08.489010: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748568428.529539   75040 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748568428.541415   75040 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-30 01:27:08.579982: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Бот запущен!
sys:1: RuntimeWarning: coroutine 'Updater.stop' was never awaited


In [1]:
!kill 222


/bin/bash: line 1: kill: (222) - No such process
